## Oracle Cloud Infrastructure 生成式 AI

Oracle Cloud Infrastructure (OCI) Generative AI 是一项完全托管的服务，提供一套最先进、可定制的大型语言模型 (LLM)，涵盖广泛的用例，并可通过单一 API 访问。
使用 OCI Generative AI 服务，您可以访问现成的预训练模型，或者在专用 AI 集群上基于您自己的数据创建和托管您自己微调的定制模型。服务的详细文档和 API 可通过 __[此链接](https://docs.oracle.com/en-us/iaas/Content/generative-ai/home.htm)__ 和 __[此链接](https://docs.oracle.com/en-us/iaas/api/#/en/generative-ai/20231130/)__ 获取。

本 Notebook 将介绍如何将 OCI 的 Generative AI 完整模型与 LangChain 结合使用。

## 设置
确保已安装 oci sdk 和 langchain-community 包

In [ ]:
!pip install -U oci langchain-community

## 用法

In [ ]:
from langchain_community.llms.oci_generative_ai import OCIGenAI

llm = OCIGenAI(
    model_id="cohere.command",
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id="MY_OCID",
    model_kwargs={"temperature": 0, "max_tokens": 500},
)

response = llm.invoke("Tell me one fact about earth", temperature=0.7)
print(response)

#### 链式调用 Prompt 模板

In [ ]:
from langchain_core.prompts import PromptTemplate

llm = OCIGenAI(
    model_id="cohere.command",
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id="MY_OCID",
    model_kwargs={"temperature": 0, "max_tokens": 500},
)

prompt = PromptTemplate(input_variables=["query"], template="{query}")
llm_chain = prompt | llm

response = llm_chain.invoke("what is the capital of france?")
print(response)

#### 流式传输

In [ ]:
llm = OCIGenAI(
    model_id="cohere.command",
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id="MY_OCID",
    model_kwargs={"temperature": 0, "max_tokens": 500},
)

for chunk in llm.stream("Write me a song about sparkling water."):
    print(chunk, end="", flush=True)

## 认证
LlamaIndex 支持的认证方法等同于其他 OCI 服务所使用的认证方法，并且遵循 __[标准 SDK 认证](https://docs.oracle.com/en-us/iaas/Content/API/Concepts/sdk_authentication_methods.htm)__ 方法，特别是 API 密钥、会话令牌、实例主体和资源主体。

API 密钥是上述示例中使用的默认认证方法。以下示例演示了如何使用不同的认证方法（会话令牌）

In [ ]:
llm = OCIGenAI(
    model_id="cohere.command",
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id="MY_OCID",
    auth_type="SECURITY_TOKEN",
    auth_profile="MY_PROFILE",  # replace with your profile name
    auth_file_location="MY_CONFIG_FILE_LOCATION",  # replace with file location where profile name configs present
)

## 专用 AI 集群
要访问托管在专用 AI 集群中的模型，请__创建一个端点__（其分配的 OCID，目前以“ocid1.generativeaiendpoint.oc1.us-chicago-1”为前缀）并将其用作您的模型 ID。

访问托管在专用 AI 集群中的模型时，您需要使用两个额外的必需参数（“provider”和“context_size”）来初始化 OCIGenAI 接口。

In [ ]:
llm = OCIGenAI(
    model_id="ocid1.generativeaiendpoint.oc1.us-chicago-1....",
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id="DEDICATED_COMPARTMENT_OCID",
    auth_profile="MY_PROFILE",  # replace with your profile name,
    auth_file_location="MY_CONFIG_FILE_LOCATION",  # replace with file location where profile name configs present
    provider="MODEL_PROVIDER",  # e.g., "cohere" or "meta"
    context_size="MODEL_CONTEXT_SIZE",  # e.g., 128000
)